<a href="https://colab.research.google.com/github/ImaginationX4/Path_to_MARL/blob/master/Deep_Q_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dueling Network ⚔️ Model

Model we build has 3 parts:

<ol>
  <li>common CNN</li>
  <li>fully connected layers for Sate/Action-value</li>
  <li>output :Q-Value</li>
</ol>





In [27]:
#credit: https://nn.labml.ai/rl/dqn/model.html
import torch
from torch import nn

class  Model(nn.Module):
  def __init__(self):
    super().__init__()
    #CNN
    self.conv = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=32, kernel_size=8, stride=4),
            # W2 = (W1 - F + 2*P) / S + 1
            # H2 = (H1 - F + 2*P) / S + 1
            nn.ReLU(),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2),
            nn.ReLU(),

            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
            nn.ReLU() #7x7
        )
    #flatten layer -->512 features
    self.lin = nn.Linear(in_features=7 * 7 * 64, out_features=512)
    self.activation = nn.ReLU()

    #V
    self.state_value = nn.Sequential(
         nn.Linear(in_features=512, out_features=256),
         nn.ReLU(),
         nn.Linear(in_features=256, out_features=1),
    )
    #A
    self.action_value = nn.Sequential(
         nn.Linear(in_features=512, out_features=256),
         nn.ReLU(),
         nn.Linear(in_features=256, out_features=4),
    )

  def forward(self, x: torch.Tensor):
    x=self.conv(x)
    x=x.reshape((-1, 7 * 7 * 64))
    x=self.activation(self.lin(x))
    action_value = self.action_value(x)
    state_value = self.state_value(x)
    action_score_centered = action_value - action_value.mean(dim=-1, keepdim=True)
    q = state_value + action_score_centered
    return q




In [33]:
m=Model()
input=torch.ones(size=(32,4, 84, 84))
output=m(input)
output

tensor([[-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192, -0.0244],
        [-0.0336,  0.0079,  0.0192

# Prioritized Experience Replay Buffer

In [ ]:
[float('inf') for _ in range(2 * 3)]
a=2
a//= 2#floor
a

1

<h2>Compute importance-sampling weight
</h2>
<p>If you dont know segment tree(Complete Binary Tree/Heap), check <a href='https://www.youtube.com/watch?v=xztU7lmDLv8'>this video</a> on Youtube to gain a better understding</p>
<p>In our case we start indexing at 1</p>

In [51]:
##Let's understnd some funtions/methods
import random
import numpy as np
#we need to add all info to the Buffer, and what's kind of data structures we use?
#Queue!
data= {
        'obs': np.zeros(shape=(32, 4, 84, 84), dtype=np.uint8),
        'action': np.zeros(shape=32, dtype=np.int32),
        'reward': np.zeros(shape=32, dtype=np.float32),
        'next_obs': np.zeros(shape=(32, 4, 84, 84), dtype=np.uint8),
        'done': np.zeros(shape=32, dtype=np.bool_)
    }

def add(obs, action, reward, next_obs, done,idx,data):
  '''
  obs is infomation gained from the env to predict the Q-value
  done means game is over or not?
  idx index
  data place to stare
  '''
  data['obs'][idx] = obs
  data['action'][idx] = action
  data['reward'][idx] = reward
  data['next_obs'][idx] = next_obs
  data['done'][idx] = done

  next_idx= (idx+1) % data['obs'].shape[0]

  return (data['obs'][idx],data['action'][idx],data['reward'][idx],data['next_obs'][idx],data['done'][idx])

obs= np.ones(shape=(4, 84, 84), dtype=np.uint8)
action= 2
reward = -999
next_obs = np.ones(shape=(4, 84, 84), dtype=np.uint8)
done = False
print(add(obs, action, reward, next_obs, done,1,data))


(array([[[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]],

       [[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]],

       [[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]],

       [[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]]], dtype=uint8), 2, -999.0, array([[[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1

In [45]:
#we try sample some data form buffer
def sample(batch_size, beta):
  samples = {
        'weights': np.zeros(shape=batch_size, dtype=np.float32),
        'indexes': np.zeros(shape=batch_size, dtype=np.int32)
    }
  for i in range(batch_size):
    samples['indexes'][i]=np.random.choice(32, 1)
  prob_min = 0.2/4 #min/sum
  max_weight = (prob_min * 32) ** (-beta)#max w
  for i in range(batch_size):
    idx = samples['indexes'][i]
    prob = 0.2/4 #self/sum
    weight = (prob * 32) ** (-beta)#weight without max w
    samples['weights'][i] = weight / max_weight
  for k, v in data.items():
    samples[k] = v[samples['indexes']]
  return samples
batch_size =63
beta=0.2
samples=sample(batch_size, beta)

<ipython-input-45-3df89bfc82ef>:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  samples['indexes'][i]=np.random.choice(32, 1)


In [6]:
#credit: https://nn.labml.ai/rl/dqn/replay_buffer.html
import random
import numpy as np

class ReplayBuffer:
  def __init__(self, capacity, alpha):#alpha high: only cares the important low: all the same
    self.capacity = capacity
    self.alpha = alpha

    self.priority_sum = [0 for _ in range(2 * self.capacity)]#sement tree to get sum of
    #float('inf') as an integer to represent it as infinity
    self.priority_min = [float('inf') for _ in range(2 * self.capacity)]

    #Current max priority, p, to be assigned to new transitions
    self.max_priority = 1.#p-value

    self.data = {
        'obs': np.zeros(shape=(capacity, 4, 84, 84), dtype=np.uint8),
        'action': np.zeros(shape=capacity, dtype=np.int32),
        'reward': np.zeros(shape=capacity, dtype=np.float32),
        'next_obs': np.zeros(shape=(capacity, 4, 84, 84), dtype=np.uint8),
        'done': np.zeros(shape=capacity, dtype=np.bool)
    }

    #cyclic buffers: FIFO
    self.next_idx = 0
    #Size of the buffer
    self.size = 0

  def add(self, obs, action, reward, next_obs, done):
    idx = self.next_idx #where to put?

    #store
    self.data['obs'][idx] = obs
    self.data['action'][idx] = action
    self.data['reward'][idx] = reward
    self.data['next_obs'][idx] = next_obs
    self.data['done'][idx] = done

    #update index of in and out
    self.next_idx = (idx + 1) % self.capacity
    self.size = min(self.capacity, self.size + 1)

    priority_alpha = self.max_priority ** self.alpha
    self._set_priority_min(idx, priority_alpha)
    self._set_priority_sum(idx, priority_alpha)

  #Set priority in binary segment tree and return minimum
  def _set_priority_min(self, idx, priority_alpha):
    idx += self.capacity
    self.priority_min[idx] = priority_alpha

    while idx >= 2:
      idx //= 2
      self.priority_min[idx] = min(self.priority_min[2 * idx], self.priority_min[2 * idx + 1])

  #Set priority in binary segment tree and return sum
  def _set_priority_sum(self, idx, priority):
    idx += self.capacity
    self.priority_sum[idx] = priority

    while idx >= 2:
      idx //= 2
      self.priority_sum[idx]= self.priority_sum[2 * idx] + self.priority_sum[2 * idx + 1]

  def _sum(self):
    return self.priority_sum[1]

  def _min(self):
    return self.priority_min[1]

  #Find i, that p0+...pi >= prefix_sum
  def find_prefix_sum_idx(self, prefix_sum):
    idx = 0
    #find the index on the leaf
    while idx < self.capacity:
      if self.priority_sum[idx * 2] > prefix_sum:
        idx = 2 * idx
      else:
        prefix_sum -= self.priority_sum[idx * 2]
        idx = 2 * idx + 1
    return idx - self.capacity

  #Sample from buffer
  def sample(self, batch_size, beta):
    samples = {
        'weights': np.zeros(shape=batch_size, dtype=np.float32),
        'indexes': np.zeros(shape=batch_size, dtype=np.int32)
    }
    #set indexes
    for i in range(batch_size):
      p = random.random() * self._sum()#random priority
      idx = self.find_prefix_sum_idx(p)
      samples['indexes'][i] = idx
     #get the max w
    prob_min = self._min() / self._sum()
    max_weight = (prob_min * self.size) ** (-beta)#max w

    #GET weight
    for i in range(batch_size):
      idx = samples['indexes'][i]
      prob = self.priority_sum[idx + self.capacity] / self._sum()
      weight = (prob * self.size) ** (-beta)
      samples['weights'][i] = weight / max_weight
    #add the rest data in samples
    for k, v in self.data.items():
      samples[k] = v[samples['indexes']]
    return samples

    def update_priorities(self, indexes, priorities):
      for idx, priority in zip(indexes, priorities):
        self.max_priority = max(self.max_priority, priority)
        priority_alpha = priority ** self.alpha

        self._set_priority_min(idx, priority_alpha)
        self._set_priority_sum(idx, priority_alpha)
    def is_full(self):
      return self.capacity == self.size

# Deep Q Networks (DQN)

<p>For now we only got the model,but we still need to loss function to calculate the loss</p>
<p>In order to do that,we should get the 'real' q  by using bellman equations, and compare it with model_q_value </p>

In [ ]:
!pip install labml-nn

In [10]:
import torch

t = torch.tensor([[1, 2], [3, 4]])
print(f"t:   {t}")
#pick elements form row/column and form a new matrix
torch.gather(t, -1, torch.tensor([[0, 1,1], [1, 0,1]]))

t:   tensor([[1, 2],
        [3, 4]])


tensor([[1, 2, 2],
        [4, 3, 4]])

<p>First get the Q(s,a) from the output of model and corresponding's action</p><p>Second we need the q(s',a),it will be given by double  Q-Learning</p>
<p>Third calculate the loss and TD-error</p>

In [44]:
import torch
from torch import nn
def forward(q: torch.Tensor, action: torch.Tensor, double_q_next: torch.Tensor,
              target_q_next: torch.Tensor, done: torch.Tensor, reward: torch.Tensor,
              weights: torch.Tensor):
  q_action = q.gather(-1, action.to(torch.long).unsqueeze(-1)).squeeze(-1)
  with torch.no_grad():
    best_next_action = torch.argmax(double_q_next, -1)
    best_next_q_value = target_q_next.gather(-1, best_next_action.unsqueeze(-1)).squeeze(-1)
    q_update = reward + 0.1 * best_next_q_value * (1 - done)
    td_error = q_action - q_update
  huber_loss = nn.SmoothL1Loss(reduction='none')
  losses = huber_loss(q_action, q_update)
  loss = torch.mean(weights * losses)
  return loss,td_error


q= output
action= torch.from_numpy(np.random.choice(4, 32))
double_q_next =Model()
double_q_next=double_q_next(torch.rand(size=(32,4,84,84)))
target_q_next = Model()
target_q_next =target_q_next(torch.rand(size=(32,4,84,84)))
done=False
reward=torch.ones(32)
weights = torch.rand(32)
forward(q, action, double_q_next,
              target_q_next, done, reward,
              weights)

(tensor(0.2524, grad_fn=<MeanBackward0>),
 tensor([-0.9737, -0.9736, -1.0169, -1.0171, -1.0265, -1.0263, -0.9847, -1.0172,
         -1.0169, -0.9736, -0.9849, -1.0266, -0.9851, -0.9849, -0.9737, -0.9733,
         -0.9847, -0.9737, -0.9850, -1.0263, -0.9850, -1.0172, -1.0171, -0.9848,
         -1.0259, -0.9848, -0.9848, -0.9734, -0.9734, -1.0265, -0.9736, -0.9851]))

In [48]:
# https://nn.labml.ai/rl/dqn/model.html
from typing import Tuple
import torch
from torch import nn


class QFuncLoss(Model):
  def __init__(self, gamma: float):
    super().__init__()
    self.gamma = gamma
    self.huber_loss = nn.SmoothL1Loss(reduction='none')
  def forward(self, q: torch.Tensor, action: torch.Tensor, double_q: torch.Tensor,
              target_q: torch.Tensor, done: torch.Tensor, reward: torch.Tensor,
              weights: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:

              q_sampled_action = q.gather(-1, action.to(torch.long).unsqueeze(-1)).squeeze(-1)
              #tracker.add('q_sampled_action', q_sampled_action)
              with torch.no_grad():
                #use double to choose action, traget uses ations to get Q-value
                best_next_action = torch.argmax(double_q, -1)
                best_next_q_value = target_q.gather(-1, best_next_action.unsqueeze(-1)).squeeze(-1)

                #update
                q_update = reward + self.gamma * best_next_q_value * (1 - done)
                #tracker.add('q_update', q_update)

                #TD
                td_error = q_sampled_action - q_update
                #tracker.add('td_error', td_error)
              losses = self.huber_loss(q_sampled_action, q_update)
              loss = torch.mean(weights * losses)
              #tracker.add('loss', loss)
              return td_error, loss


# Game environment

Not really understnd it....

In [ ]:
#credit: https://nn.labml.ai/rl/game.html
from multiprocessing import Process
import os
import multiprocessing

def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(a):
    a=a.recv()
    info('function f')
    print('hello', "what's up",a)

child, parent = multiprocessing.Pipe()
p = Process(target=f, args=(parent,))
child.send('AAHPH')
p.start()

function f
module name: __main__
parent process: 4228
process id: 4343
hello what's up 
AAHPH

In [ ]:
import multiprocessing
import multiprocessing.connection

import cv2
import gym
import numpy as np

class Game:
  def __init__(self, seed: int):
    self.env = gym.make('BreakoutNoFrameskip-v4')
    self.env.seed(seed)#making experiments reproducible.

    self.obs_4 = np.zeros((4, 84, 84))#a stack of 4 frames
    self.obs_2_max = np.zeros((2, 84, 84))#buffer to keep the maximum of last 2 frames

    self.rewards = []
    self.lives = 0
  def step(self, action):
    reward = 0.
    done = None

    for i in range(4):
      obs, r, done, info = self.env.step(action)
      if i >= 2:
        self.obs_2_max[i % 2] = self._process_obs(obs)#proceed img
      reward += r
      lives = self.env.unwrapped.ale.lives()
      if lives < self.lives:#if a life is lost
        done = True
        break
    self.rewards.append(reward)

    if done:
      episode_info = {"reward": sum(self.rewards), "length": len(self.rewards)}
      self.reset()
    else:
      episode_info = None
      obs = self.obs_2_max.max(axis=0)
      self.obs_4 = np.roll(self.obs_4, shift=-1, axis=0)#move 1 step left
      self.obs_4[-1] = obs#replacce the oldest
    return self.obs_4, reward, done, episode_info

  def reset(self):
    obs = self.env.reset()
    #reset caches
    obs = self._process_obs(obs)
    for i in range(4):
      self.obs_4[i] = obs
    self.rewards = []
    self.lives = self.env.unwrapped.ale.lives()

  @staticmethod
  def _process_obs(obs):
    obs = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)
    obs = cv2.resize(obs, (84, 84), interpolation=cv2.INTER_AREA)
    return obs

def worker_process(remote: multiprocessing.connection.Connection, seed: int):

  game = Game(seed)
  while True:
    cmd, data = remote.recv()
    if cmd == "step":
      remote.send(game.step(data))
    elif cmd == "reset":
      remote.send(game.reset())
    elif cmd == "close":
      remote.close()
      break
    else:
      raise NotImplementedError
class Worker:
  def __init__(self, seed):
    self.child, parent = multiprocessing.Pipe()#send--->recieve
    self.process = multiprocessing.Process(target=worker_process, args=(parent, seed))
    self.process.start()

# DQN Experiment with Atari Breakout

In [ ]:
!pip3 install mxnet-mkl==1.6.0 numpy==1.23.1

<p>I try to train our model, but i dont really understnd the environment,so i assume i've alredy have the buffer info</p>

<p></p>

In [54]:
import numpy as np
import torch
def obs_to_torch(obs: np.ndarray) -> torch.Tensor:

    return torch.tensor(obs, dtype=torch.float32)
EPOCHS=100
model = Model()
target_model = Model()
loss_func = QFuncLoss(0.99)
optimizer = torch.optim.Adam(model.parameters(), lr=2.5e-4)
def train():
  for epoch in range(EPOCHS):
    model.train()
    #q(s,a)
    q_value = model(obs_to_torch(samples['obs']))
    #q(s',a)
    with torch.inference_mode():
      double_q_value = model(obs_to_torch(samples['next_obs']))
      target_q_value =target_model(obs_to_torch(samples['next_obs']))
    #Loss
    td_errors, loss =loss_func(q_value,
                  q_value.new_tensor(samples['action']),
                  double_q_value, target_q_value,
                  q_value.new_tensor(samples['done']),
                  q_value.new_tensor(samples['reward']),
                  q_value.new_tensor(samples['weights']))
    new_priorities = td_errors
    #clean record
    optimizer.zero_grad()
    #gradient
    loss.backward()
    #change the setting
    optimizer.step()
    if epoch%10==0:
      print(f'epoch:{epoch}---train_loss:{loss}')
train()

epoch:0---train_loss:31.711091995239258
epoch:10---train_loss:31.508651733398438
epoch:20---train_loss:27.719820022583008
epoch:30---train_loss:6.995792388916016
epoch:40---train_loss:6.931807994842529
epoch:50---train_loss:4.018191814422607
epoch:60---train_loss:0.04735827073454857
epoch:70---train_loss:0.4734249413013458
epoch:80---train_loss:0.1565019190311432
epoch:90---train_loss:0.6465544104576111


In [ ]:
#credit: https://nn.labml.ai/rl/dqn/experiment.html
import numpy as np
import torch
from labml import tracker, experiment, logger, monit
from labml.internal.configs.dynamic_hyperparam import FloatDynamicHyperParam
from labml_helpers.schedule import Piecewise

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
def obs_to_torch(obs: np.ndarray) -> torch.Tensor:
  return torch.tensor(obs, dtype=torch.float32, device=device) / 255.
class Trainer:
  def __init__(self, *,updates: int, epochs: int,
              n_workers: int, worker_steps: int, mini_batch_size: int,
              update_target_model: int,
              learning_rate: FloatDynamicHyperParam,):
    self.n_workers = n_workers
    self.worker_steps = worker_steps
    self.train_epochs = epochs
    self.updates = updates
    self.mini_batch_size = mini_batch_size
    self.update_target_model = update_target_model
    self.learning_rate = learning_rate
    self.exploration_coefficient = Piecewise([
               (0, 1.0),
               (25_000, 0.1),
               (self.updates / 2, 0.01)], outside_value=0.01)
    self.prioritized_replay_beta = Piecewise([
                (0, 0.4),
                (self.updates, 1)], outside_value=1)
    self.replay_buffer = ReplayBuffer(2 ** 14, 0.6)
    self.model = Model().to(device)
    self.target_model = Model().to(device)
    self.workers = [Worker(47 + i) for i in range(self.n_workers)]
    # initialize tensors for observations
    self.obs = np.zeros((self.n_workers, 4, 84, 84), dtype=np.uint8)

    # reset the workers
    for worker in self.workers:
      worker.child.send(("reset", None))

    # get the initial observations
    for i, worker in enumerate(self.workers):
      self.obs[i] = worker.child.recv()

    # loss function
    self.loss_func = QFuncLoss(0.99)
    # optimizer
    self.optimizer = torch.optim.Adam(self.model.parameters(), lr=2.5e-4)

  def _sample_action(self, q_value: torch.Tensor, exploration_coefficient: float):
    with torch.no_grad():
      greedy_action = torch.argmax(q_value, dim=-1)
      random_action = torch.randint(q_value.shape[-1], greedy_action.shape, device=q_value.device)
      is_choose_rand = torch.rand(greedy_action.shape, device=q_value.device) < exploration_coefficient
      return torch.where(is_choose_rand, random_action, greedy_action).cpu().numpy()

  def sample(self, exploration_coefficient: float):
    with torch.no_grad():
      for t in range(self.worker_steps):
        q_value = self.model(obs_to_torch(self.obs))
        actions = self._sample_action(q_value, exploration_coefficient)
        for w, worker in enumerate(self.workers):
          worker.child.send(("step", actions[w]))
        for w, worker in enumerate(self.workers):
          next_obs, reward, done, info = worker.child.recv()
          self.replay_buffer.add(self.obs[w], actions[w], reward, next_obs, done)
          if info:
            tracker.add('reward', info['reward'])
            tracker.add('length', info['length'])
          self.obs[w] = next_obs
  def train(self, beta: float):
    for _ in range(self.train_epochs):
      samples = self.replay_buffer.sample(self.mini_batch_size, beta)
      q_value = self.model(obs_to_torch(samples['obs']))
      with torch.no_grad():
        double_q_value = self.model(obs_to_torch(samples['next_obs']))
        target_q_value = self.target_model(obs_to_torch(samples['next_obs']))
      #Compute Temporal Difference (TD) errors, δ, and the loss, L(θ)
      td_errors, loss = self.loss_func(q_value,q_value.new_tensor(samples['action']),
                         double_q_value, target_q_value,
                        q_value.new_tensor(samples['done']),
                        q_value.new_tensor(samples['reward']),
                        q_value.new_tensor(samples['weights']))
      new_priorities = np.abs(td_errors.cpu().numpy()) + 1e-6
      self.replay_buffer.update_priorities(samples['indexes'], new_priorities)
      for pg in self.optimizer.param_groups:
        pg['lr'] = self.learning_rate()

      self.optimizer.zero_grad()
      loss.backward()
      torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=0.5)
      self.optimizer.step()

    def run_training_loop(self):
       tracker.set_queue('reward', 100, True)
       tracker.set_queue('length', 100, True)

       self.target_model.load_state_dict(self.model.state_dict())

       for update in monit.loop(self.updates):
        exploration = self.exploration_coefficient(update)
        tracker.add('exploration', exploration)

        beta = self.prioritized_replay_beta(update)
        tracker.add('beta', beta)
        self.sample(exploration)

        if self.replay_buffer.is_full():
          self.train(beta)
          if update % self.update_target_model == 0:
            self.target_model.load_state_dict(self.model.state_dict())
        tracker.save()
        if (update + 1) % 1_000 == 0:
          logger.log()
    def destroy(self):
        """
        ### Destroy
        Stop the workers
        """
        for worker in self.workers:
            worker.child.send(("close", None))


def main():

  # Create the experiment
  experiment.create(name='dqn')

  # Configurations
  configs = {
        # Number of updates
        'updates': 1_000_000,
        # Number of epochs to train the model with sampled data.
        'epochs': 8,
        # Number of worker processes
        'n_workers': 8,
        # Number of steps to run on each process for a single update
        'worker_steps': 4,
        # Mini batch size
        'mini_batch_size': 32,
        # Target model updating interval
        'update_target_model': 250,
        # Learning rate.
        'learning_rate': FloatDynamicHyperParam(1e-4, (0, 1e-3)),
    }

  # Configurations
  experiment.configs(configs)

  # Initialize the trainer
  m = Trainer(**configs)
  # Run and monitor the experiment
  with experiment.start():
    m.run_training_loop()
  # Stop the workers
  m.destroy()


# ## Run it
if __name__ == "__main__":
    main()




<ipython-input-3-a4a2a07a63f4>:21: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  'done': np.zeros(shape=capacity, dtype=np.bool)
Process Process-10:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Process Process-12:
Process Process-11:
  File "<ipython-input-8-af899276e9fd>", line 60, in worker_process
    game = Game(seed)
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-13:
  File "/usr/lib/python3.10/multiprocessing/process.py"

BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


class LinearModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(10, 20)
        self.layer2 = nn.Linear(20, 3)
        self.layer3 = nn.Linear(3, 1)

    def forward(self, x):
        return self.layer3(self.layer2(self.layer1(x)))



model = LinearModel()
optimizer = optim.SGD(model.parameters(),0.02)
for lr in optimizer.param_groups:
  lr['lr'] = 1111
optimizer.param_groups

In [ ]:
def sample_action( q_value: torch.Tensor, exploration_coefficient: float):
  with torch.no_grad():
    greedy_action = torch.argmax(q_value, dim=-1)
    random_action = torch.randint(q_value.shape[-1], greedy_action.shape, device=q_value.device)
    is_choose_rand = torch.rand(greedy_action.shape, device=q_value.device) < exploration_coefficient
    return torch.where(is_choose_rand, random_action, greedy_action).cpu().numpy()
a=torch.tensor(
    [1,2,3,4])
sample_action(a,2.2)